In [1]:
import pandas as pd
import geopandas as gpd
import fiona
import leafmap.foliumap as leafmap

In [2]:
path_row = "data/fiber_lines.gpkg"
path_customers = "data/coverage_streets_raw.gpkg"

Load layers in `Right of Way` file

In [3]:
# Load all layers from right_of_way.gpkg
layers = fiona.listlayers(path_row)
gdf = [gpd.read_file(path_row, layer=lyr) for lyr in layers]
layers

['Area_1', 'Area_2', 'Area_3', 'Area_4']

Merge all the layers in the `Right of Way` package

In [4]:
# Merge them into one GeoDataFrame
gdf_right = gpd.GeoDataFrame(pd.concat(gdf, ignore_index=True), crs=gdf[0].crs)

Read `Customer coverage` data

In [5]:
# Load all lines
gdf_all = gpd.read_file(path_customers)

# Make sure they’re in the same CRS
gdf_all = gdf_all.to_crs(gdf_right.crs)

- Right of Way has 699 LineStrings
- Total Customer Coverage has 4522 LineStrings

In [6]:
gdf_all["geometry"]

0       MULTILINESTRING ((539051.588 734446.242, 53906...
1       MULTILINESTRING ((539348.629 735273.988, 53930...
2       MULTILINESTRING ((538783.354 735536.111, 53877...
3       MULTILINESTRING ((538652.132 734970.537, 53872...
4       MULTILINESTRING ((538667.543 735013.782, 53867...
                              ...                        
4517    MULTILINESTRING ((546468.623 711304.651, 54648...
4518    MULTILINESTRING ((547245.666 712127.567, 54724...
4519    MULTILINESTRING ((546565.5 711536.769, 546563....
4520    MULTILINESTRING ((546356.942 711486.078, 54635...
4521    MULTILINESTRING ((540988.433 711969.593, 54104...
Name: geometry, Length: 4522, dtype: geometry

Break down MultiLineStrings into LineStrings so that the lines that can iterated individually

In [7]:
gdf_right = gdf_right.explode(index_parts=False, ignore_index=True)
gdf_all = gdf_all.explode(index_parts=False, ignore_index=True)

Check for lines that intersect between the Right of Way and Customer Coveragw and filter them out
- The lines may be really close and not perfectly intersect, so we need a buffer for 5 metres

In [8]:
# Option 1: exact geometry comparison

buffered_right = gdf_right.buffer(39)  # 5 meters tolerance
# gdf_non_right = gdf_all[~gdf_all.intersects(buffered_right.union_all())]
gdf_non_right = gdf_all[~gdf_all.intersects(gdf_right.unary_union)]
gdf_non_right["geometry"].value_counts().sum()

np.int64(2967)

There are 2967 LineStrings without Right of Way
- 4522 -> 2967() -> 2844(buffer of 5m) -> 2594(buffer off 39m)

Calculate distance and assign values to a field

In [9]:
gdf_non_right = gdf_non_right.to_crs(epsg=32631)
gdf_non_right["distance_m"] = gdf_non_right.geometry.length

Plot the Customer Coverage without Right of Way

In [10]:
# Create a map centered roughly on your data
m = leafmap.Map(center=[6.45, 3.39], zoom=9, style="streets")

# Style for non-right-of-way lines
style_non_right = {
    "color": "blue",
    "weight": 2,
}

# Style for right-of-way lines
style_right = {
    "color": "red",
    "weight": 2,
}

# Add the GeoDataFrame
# tooltip_fields = [col for col in ["road_street_name", "LOCAL GOVERNMENT", "distance(m)"] if col in gdf_non_right.columns]

# Add non-right-of-way roads
m.add_gdf(
    gdf_non_right,
    layer_type="line",
    layer_name="Non-Right-of-Way Roads",
    style=style_non_right,
)

# Add right-of-way roads
m.add_gdf(
    gdf_right,
    layer_type="line",
    layer_name="Right-of-Way Roads",
    style=style_right,
)

# Zoom to fit both datasets
m.zoom_to_gdf(pd.concat([gdf_non_right, gdf_right]))

m

Browser

In [ ]:
# webbrowser.open("map.html")

Save File

In [ ]:
gdf_non_right.to_file("Customers_without_right_of_way.gpkg", driver="GPKG")